# Analyse simple du risque de ré‑identification (sur **données simulées**)

**Objectif** : montrer, avec des données simulées, comment on peut évaluer le risque que des personnes soient identifiables dans un jeu de données en regardant la **taille des groupes** qui partagent la même
combinaison de caractéristiques (âge, sexe, codes médicaux, etc.).

**Idée clé** : si une combinaison n'appartient qu'à 1 seule personne (`k = 1`),
alors cette personne est *ré‑identifiable* par quelqu'un qui connaît cette combinaison.


## Ce que contient ce notebook
1. **On simule** un jeu de données simple avec 100 000 personnes.
2. On calcule les **tailles de groupes** (k) pour différentes **granularités** (plus ou moins détaillées).
3. On affiche des **tableaux** et **histogrammes** pour voir le risque.
4. On montre l'effet de **coarsening** (regrouper des catégories) et de **retirer un axe** très discriminant.


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1) Paramètres de simulation

On simule :
- 20 tranches d'âge (classes de 5 ans),
- 2 sexes,
- 2 valeurs d'issue (transfusion oui/non, ici ~1%),
- **CIM‑10** : 22 chapitres,
- **ATC** : 14 grandes classes,
- **CCAM** : 19 chapitres.

Tu peux modifier ces paramètres ci‑dessous pour tester d'autres cas.


In [ ]:
# Pour des résultats reproductibles
np.random.seed(42)

# Taille de la population simulée
N = 100_000

# Espaces de catégories
AGE_BUCKETS_20 = np.arange(20)     # 20 classes d'âge (0..19)
SEXES = np.array(['F', 'H'])       # 2 sexes
TRANSFUSION_RATE = 0.01            # 1% de transfusion

N_CIM10 = 22                        # chapitres CIM-10
N_ATC = 14                          # classes ATC (niveau agrégé)
N_CCAM = 19                         # chapitres CCAM

def zipf_probs(k: int, s: float = 1.1) -> np.ndarray:
    """Renvoie une loi 'type Zipf' (les premières catégories sont plus fréquentes)."""
    ranks = np.arange(1, k + 1)
    p = 1.0 / (ranks ** s)
    return p / p.sum()


## 2) Simulation d'un jeu de données

On génère des colonnes simples :
- `age_bucket20`, `sex`, `transfusion`
- `icd_chap` (CIM‑10), `atc_grp` (ATC), `ccam_chap` (CCAM)

On choisit des distributions un peu **déséquilibrées** (certaines catégories sont plus fréquentes),
pour imiter la réalité où tout n'est pas uniformément réparti.


In [ ]:
# Probabilités (catégories déséquilibrées, plus réalistes qu'un tirage uniforme)
p_icd = zipf_probs(N_CIM10, s=1.1)
p_atc = zipf_probs(N_ATC,   s=1.0)
p_ccam = zipf_probs(N_CCAM, s=1.15)

# Tirages aléatoires
age = np.random.choice(AGE_BUCKETS_20, size=N, replace=True)
sex = np.random.choice(SEXES, size=N, replace=True)
transfusion = (np.random.rand(N) < TRANSFUSION_RATE).astype(int)

icd = np.random.choice(np.arange(1, N_CIM10 + 1), size=N, p=p_icd)
atc = np.random.choice(np.arange(1, N_ATC + 1),   size=N, p=p_atc)
ccam = np.random.choice(np.arange(1, N_CCAM + 1), size=N, p=p_ccam)

df = pd.DataFrame({
    'age_bucket20': age,
    'sex': sex,
    'transfusion': transfusion,
    'icd_chap': icd,
    'atc_grp': atc,
    'ccam_chap': ccam,
})

df.head()

## 3) Fonction utilitaire : tailles de groupes (k)

On compte combien de personnes partagent **exactement** la même combinaison de colonnes choisies.


In [ ]:
from typing import List, Tuple, Dict

def group_size_table(data: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """Return a table with one row per combination and a 'count' column = group size (k)."""
    g = data.value_counts(subset=cols).reset_index(name='count')
    return g

def k_stats_for_reporting(g: pd.DataFrame, label: str) -> Dict[str, float]:
    """Return simple stats to summarize risk in plain language."""
    observed = len(g)  # number of combos that actually appear
    uniques = (g['count'] == 1).sum()
    lt5 = (g['count'] < 5).sum()
    # Patient-weighted stats
    total = int(g['count'].sum())
    patients_k1 = int(g.loc[g['count'] == 1, 'count'].sum())
    patients_klt5 = int(g.loc[g['count'] < 5, 'count'].sum())
    # Weighted median of k
    median_k_weighted = float(np.median(np.repeat(g['count'].values, g['count'].values)))
    return {
        'setting': label,
        'observed_combos': int(observed),
        'unique_combos': int(uniques),
        'prop_unique_combos': uniques / observed if observed else np.nan,
        'prop_combos_k<5': lt5 / observed if observed else np.nan,
        'patients_total': total,
        'patients_in_k=1': patients_k1,
        'prop_patients_k=1': patients_k1 / total if total else np.nan,
        'patients_in_k<5': patients_klt5,
        'prop_patients_k<5': patients_klt5 / total if total else np.nan,
        'median_k_weighted': median_k_weighted,
    }

## 4) Granularité **fine** (combinatoire détaillée)

Clé d'équivalence utilisée :
```
(age_bucket20, sex, transfusion, icd_chap, atc_grp, ccam_chap)
```
Plus on met de détails, plus on crée de petites classes (souvent de taille 1), donc risque ↑.


In [ ]:
cols_fine = ['age_bucket20', 'sex', 'transfusion', 'icd_chap', 'atc_grp', 'ccam_chap']
g_fine = group_size_table(df, cols_fine)
stats_fine = k_stats_for_reporting(g_fine, 'fine')
pd.DataFrame([stats_fine])

In [ ]:
# Distribution des tailles de groupes pour le sous-groupe "transfusion = 1"
g_fine_tx = group_size_table(df[df['transfusion'] == 1], cols_fine)
plt.figure()
plt.hist(g_fine_tx['count'], bins=50)
plt.title('Tailles de groupes (k) — transfusion=1 — granularité fine')
plt.xlabel('k')
plt.ylabel('nombre de classes')
plt.show()

## 5) **Regrouper** des catégories (coarsening)

On fusionne des catégories pour **grossir** les groupes :
- Âge : 10 classes (au lieu de 20),
- CIM‑10 : ~10 super‑groupes,
- ATC : ~7 super‑groupes,
- CCAM : ~10 super‑groupes.

But : faire monter `k` sans tout effacer.


In [ ]:
df_c = df.copy()
# Age: 20 -> 10 classes
df_c['age_bucket10'] = df_c['age_bucket20'] // 2

# Exemple simple de fusion en "super-groupes" par paquets
df_c['icd_super10']  = ((df_c['icd_chap'] - 1) // 2 + 1).clip(1, 10)  # 22 -> ~10
df_c['atc_super7']   = ((df_c['atc_grp']  - 1) // 2 + 1)               # 14 -> 7
df_c['ccam_super10'] = ((df_c['ccam_chap']- 1) // 2 + 1)               # 19 -> ~10

cols_coarse = ['age_bucket10','sex','transfusion','icd_super10','atc_super7','ccam_super10']
g_coarse = group_size_table(df_c, cols_coarse)
stats_coarse = k_stats_for_reporting(g_coarse, 'coarse')
pd.DataFrame([stats_coarse])

In [ ]:
# Même histogramme sur le sous-groupe transfusion=1 après coarsening
g_coarse_tx = group_size_table(df_c[df_c['transfusion'] == 1], cols_coarse)
plt.figure()
plt.hist(g_coarse_tx['count'], bins=50)
plt.title('Tailles de groupes (k) — transfusion=1 — après regroupements')
plt.xlabel('k')
plt.ylabel('nombre de classes')
plt.show()

## 6) Retirer un axe très discriminant (ex.: CCAM)

On recommence sans utiliser **CCAM** dans la clé d'équivalence publique :
```
(age_bucket10, sex, transfusion, icd_super10, atc_super7)
```
L'idée n'est pas de "tricher", mais de **limiter** les détails publiés si un axe rend
quasi tout le monde unique. En pratique, on peut publier CCAM **plus agrégé** ou via des **indicateurs**.


In [ ]:
cols_coarse_no_ccam = ['age_bucket10','sex','transfusion','icd_super10','atc_super7']
g_no_ccam = group_size_table(df_c, cols_coarse_no_ccam)
stats_no_ccam = k_stats_for_reporting(g_no_ccam, 'coarse minus CCAM')
pd.DataFrame([stats_no_ccam])

## 7) Résumé comparatif

On compare les trois situations :
- **fine** (très détaillé) → risque le plus élevé,
- **coarse** (catégories regroupées) → risque en baisse,
- **coarse minus CCAM** → risque encore plus bas.

La colonne `prop_patients_k<5` est souvent la plus parlante : elle dit **quelle part des personnes**
se retrouvent dans de **toutes petites classes** (k<5).


In [ ]:
summary = pd.DataFrame([stats_fine, stats_coarse, stats_no_ccam])
summary[['setting','observed_combos','prop_patients_k=1','prop_patients_k<5','median_k_weighted']].sort_values('setting')

## 8) Comment lire ces résultats (sans jargon)
- Si beaucoup de personnes sont en **k=1** (ou **k<5**), le **risque est élevé**.
- **Regrouper** des catégories (classes d'âge plus larges, chapitres médicaux plus généraux)
  **augmente k** et **réduit** le risque.
- Si un **axe** rend presque tout le monde unique (ex. un code trop fin), on peut :
  - publier cet axe **plus agrégé**,
  - ou ne pas l'utiliser pour former les **classes d'équivalence** publiques,
  - ou **basculer vers des données 100% synthétiques** pour la publication large.


## 9) Pour aller plus loin (optionnel)
- Tester d'autres découpages (âge en 5 classes, 8 classes, etc.).
- Tester l'effet de retirer **ICD** ou **ATC** plutôt que **CCAM**.
- Ajouter un **contrôle** spécifique sur les sous‑groupes **rares** (ex. transfusion=1).
- Si publication ouverte : envisager un **jeu de données synthétiques** et documenter des tests de non‑fuite (ex. nearest‑neighbor).

**Références utiles (simples à lire) :**
- ICO (UK) — Introduction à l’anonymisation : https://ico.org.uk/for-organisations/uk-gdpr-guidance-and-resources/data-sharing/anonymisation/introduction-to-anonymisation/
- NIST SP 800‑226 (2025) — Guide d’évaluation de la confidentialité différentielle : https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-226.pdf
